In [ ]:
import pynattas as pnas

# Datamodule

In [ ]:
import datasets.SPECTRE_prova.dataset as dataset_SPECTRE
import os
import json
from PIL import Image
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms
import pytorch_lightning as pl
from pycocotools.coco import COCO
import torch
import numpy as np

In [ ]:
# Example usage
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor()
])

In [ ]:
# Create an instance of the data module with transformations
data_module = dataset_SPECTRE.SPECTRE_COCO_DataModule(img_dir=rf'D:\Andrea\DicDic\data\SPECTRE_prova\images', ann_file=rf'D:\Andrea\DicDic\data\SPECTRE_prova\annotations_SPECTRE.json', batch_size=4, transform=None)

In [ ]:
data_module

In [ ]:
data_module.setup()

In [ ]:
train_loader = data_module.train_dataloader()

In [ ]:
for images, targets in train_loader:
    print("Batch Image Shape:", images[0].shape)
    print("Batch Targets:", targets)
    break  # Inspect only the first batch

In [ ]:
import rasterio

In [ ]:
with rasterio.open(rf"D:\Andrea\DicDic\data\SPECTRE_prova\images\DAN_S1A_20240301T054857_TC_Sub1_crop1_VH_636018753.tif") as src:
    image = src.read()  # Read the single channel
    #image = np.expand_dims(image, axis=0)  # Add channel dimension to make it (height, width, 1)

print(image.shape)
#image = Image.fromarray(image)

In [ ]:
type(image)

In [ ]:
image = image[0,:,:]

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.imshow(image, cmap='gray')

In [ ]:
mean = np.mean(image)
std = np.std(image)
mask = image > (mean + 7*std)

plt.imshow(mask, cmap='gray')

# ANCORE

In [ ]:
np.array([[10, 25], 
 [15, 15], 
 [25, 10], 
 [25, 50],
 [35, 35], 
 [50, 25]])/512

# RUN

In [ ]:
%run main.py

# Testing a single run

In [1]:
import pynattas as pnas
from torchvision import transforms
import pytorch_lightning as pl
from pytorch_lightning.callbacks import EarlyStopping
import torch
from PIL import Image, ImageDraw, ImageFont
import matplotlib.pyplot as plt
import numpy as np
from datasets.SPECTRE_prova.dataset import SPECTRE_COCO_Dataset, SPECTRE_COCO_DataModule

In [2]:
root_dir='data/SPECTRE_prova'
input_size = 512
custom_transform = transforms.Compose([
    #transforms.Resize((input_size, input_size)),  # Resize images to the size expected by your model
    transforms.ToTensor(),
])
dm = SPECTRE_COCO_DataModule(
        img_dir=root_dir + '/images',
        ann_file=root_dir + '/annotations_SPECTRE.json',
        batch_size=4,
        num_workers=0,
        transform=custom_transform,
    )

In [3]:
code = pnas.functions.architecture_builder.generate_random_architecture_code(4)
print(code)
parsed_layers = pnas.functions.architecture_builder.parse_architecture_code(code)
print(parsed_layers)

model = pnas.classes.GenericOD_YOLOv3_SmallObjects(
    parsed_layers=parsed_layers,
    input_channels=1,
    input_height=512,
    input_width=512,
    num_classes=1,
    learning_rate=1e-2,
)
if model.model.is_too_deep:
    print("This architecture is too deep and the tensor lost its dimensionality")

This architecture has 3 encoder layers.
Lne4agn1EPIELpe3aln1EPMELme3arn1EPaEHSu2v4EE
[{'layer_type': 'MBConvNoRes', 'expansion_factor': 4, 'activation': 'GELU', 'num_blocks': 1}, {'layer_type': 'Identity'}, {'layer_type': 'CSPMBConvBlock', 'expansion_factor': 3, 'activation': 'LeakyReLU', 'num_blocks': 1}, {'layer_type': 'MaxPool'}, {'layer_type': 'MBConv', 'expansion_factor': 3, 'activation': 'ReLU', 'num_blocks': 1}, {'layer_type': 'AvgPool'}, {'layer_type': 'DetectionHeadYOLOv3_SmallObjects', 'outchannel1_index': 2, 'outchannel2_index': 4}]


In [5]:
# test
# Generate a random tensor with the appropriate shape
input_tensor = torch.randn(1, 1, 512, 512)  # Batch size of 1, 3 channels, 416x416 height and width

# Pass the tensor through the model
output = model(input_tensor)

# Optional: Print the output shape to verify
print("Output shape:", output.shape)

out1:  torch.Size([1, 3072, 6]) - out2:  torch.Size([1, 12288, 6]) - out:  torch.Size([1, 15360, 6])
Output shape: torch.Size([1, 15360, 6])


In [4]:
torch.set_float32_matmul_precision('medium')
trainer = pl.Trainer(
    accelerator='gpu',
    min_epochs=1,
    max_epochs=500,
    fast_dev_run=False,
    overfit_batches=True,
    check_val_every_n_epoch=10,
    callbacks=[EarlyStopping(monitor='val_loss', mode="min", patience=3)]
)
# Training
#model.device = "cuda"
#dm.device = "cuda"
trainer.fit(model, dm)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
`Trainer(overfit_batches=1)` was configured so 1 batch will be used.



loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type           | Params
-----------------------------------------
0 | model | GenericNetwork | 5.0 M 
-----------------------------------------
5.0 M     Trainable params
0         Non-trainable params
5.0 M     Total params
20.131    Total estimated model params size (MB)


Sanity Checking DataLoader 0:   0%|          | 0/True [00:00<?, ?it/s]

c:\Users\andre\anaconda3\envs\deeplearning\Lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:441: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=19` in the `DataLoader` to improve performance.


out1:  torch.Size([4, 3072, 6]) - out2:  torch.Size([4, 12288, 6]) - out:  torch.Size([4, 15360, 6])
Noobj_loss: 0.9716344475746155
Coord_loss: 9.695586204528809
Obj_loss: 0.4738689363002777
Class_loss: 0
                                                                              

c:\Users\andre\anaconda3\envs\deeplearning\Lib\site-packages\pytorch_lightning\utilities\data.py:77: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 4. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.
c:\Users\andre\anaconda3\envs\deeplearning\Lib\site-packages\pytorch_lightning\trainer\connectors\logger_connector\result.py:212: You called `self.log('val_noobj_loss', ...)` in your `validation_step` but the value needs to be floating to be reduced. Converting it to torch.float32. You can silence this warning by converting the value to floating point yourself. If you don't intend to reduce the value (for instance when logging the global step or epoch) then you can use `self.logger.log_metrics({'val_noobj_loss': ...})` instead.
c:\Users\andre\anaconda3\envs\deeplearning\Lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:268: You requested to overfit but enabled train dataloader shuffling. We are turni

Epoch 0:   0%|          | 0/1 [00:00<?, ?it/s] out1:  torch.Size([4, 3072, 6]) - out2:  torch.Size([4, 12288, 6]) - out:  torch.Size([4, 15360, 6])
Noobj_loss: 0.9709323048591614
Coord_loss: 53.6727180480957
Obj_loss: 0.4655582010746002
Class_loss: 0


c:\Users\andre\anaconda3\envs\deeplearning\Lib\site-packages\pytorch_lightning\trainer\connectors\logger_connector\result.py:212: You called `self.log('train_noobj_loss', ...)` in your `training_step` but the value needs to be floating to be reduced. Converting it to torch.float32. You can silence this warning by converting the value to floating point yourself. If you don't intend to reduce the value (for instance when logging the global step or epoch) then you can use `self.logger.log_metrics({'train_noobj_loss': ...})` instead.


Epoch 1:   0%|          | 0/1 [00:00<?, ?it/s, v_num=3937]        out1:  torch.Size([4, 3072, 6]) - out2:  torch.Size([4, 12288, 6]) - out:  torch.Size([4, 15360, 6])
Noobj_loss: 0.9099503755569458
Coord_loss: 94.41686248779297
Obj_loss: 0.3892514407634735
Class_loss: 0
Epoch 2:   0%|          | 0/1 [00:00<?, ?it/s, v_num=3937]        out1:  torch.Size([4, 3072, 6]) - out2:  torch.Size([4, 12288, 6]) - out:  torch.Size([4, 15360, 6])
Noobj_loss: 0.8234810829162598
Coord_loss: 53.38959503173828
Obj_loss: 0.410013884305954
Class_loss: 0
Epoch 3:   0%|          | 0/1 [00:00<?, ?it/s, v_num=3937]        out1:  torch.Size([4, 3072, 6]) - out2:  torch.Size([4, 12288, 6]) - out:  torch.Size([4, 15360, 6])
Noobj_loss: 0.7665918469429016
Coord_loss: 53.37739181518555
Obj_loss: 0.38342103362083435
Class_loss: 0
Epoch 4:   0%|          | 0/1 [00:00<?, ?it/s, v_num=3937]        out1:  torch.Size([4, 3072, 6]) - out2:  torch.Size([4, 12288, 6]) - out:  torch.Size([4, 15360, 6])
Noobj_loss: 0.747796